In [ ]:
from glob import glob
import numpy as np
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
squeue_files_all = glob('/project/lgrandi/yuanlq/shared/squeue_lgrandi_monitor_*.txt')
nodestatus_files_all = glob('/project/lgrandi/yuanlq/shared/nodestatus_lgrandi_monitor_*.txt')

In [ ]:
date_format = "%Y-%m-%d_%H-%M-%S"
date_times_squeue_files_all = []
notebook_jobs = []
batch_jobs = []
user_counts = []
for i in tqdm(range(len(squeue_files_all))):
    date_string = squeue_files_all[i].split('/')[-1].split('.')[0].split('squeue_lgrandi_monitor_')[-1]
    date_times_squeue_files_all.append(datetime.strptime(date_string, date_format))
    users = []
    notebooks = 0
    batches = 0
    with open(squeue_files_all[i]) as f:
        lines = f.readlines()
        for line in lines[1:]:
            line_list = line.split(' ')
            line_list = np.array(line_list)
            line_list = line_list[line_list!='']
            
            is_notebook = line_list[2]=='straxlab'
            if is_notebook:
                notebooks += 1
            else:
                batches += 1
                
            user = line_list[3]
            users.append(user)
    user_counts.append(len(np.unique(np.array(users))))
    batch_jobs.append(batches)
    notebook_jobs.append(notebooks)

date_times_squeue_files_all = np.array(date_times_squeue_files_all)
notebook_jobs = np.array(notebook_jobs)
batch_jobs = np.array(batch_jobs)
user_counts = np.array(user_counts)

notebook_jobs = notebook_jobs[np.argsort(date_times_squeue_files_all)]
batch_jobs = batch_jobs[np.argsort(date_times_squeue_files_all)]
user_counts = user_counts[np.argsort(date_times_squeue_files_all)]
date_times_squeue_files_all = np.sort(date_times_squeue_files_all)

In [ ]:
date_times_nodestatus_files_all = []
cpus_used = []
rams_used = []
for i in tqdm(range(len(nodestatus_files_all))):
    date_string = nodestatus_files_all[i].split('/')[-1].split('.')[0].split('nodestatus_lgrandi_monitor_')[-1]
    date_times_nodestatus_files_all.append(datetime.strptime(date_string, date_format))
    cpus = 0
    rams = 0
    with open(nodestatus_files_all[i]) as f:
        lines = f.readlines()
        for line in lines[5:]:
            line_list = line.split(' ')
            line_list = np.array(line_list)
            line_list = line_list[line_list!='']
            
            cpus += eval(line_list[5])
            rams += eval(line_list[7][:-2])
                
    cpus_used.append(cpus)
    rams_used.append(rams)

date_times_squeue_files_all = np.array(date_times_squeue_files_all)
cpus_used = np.array(cpus_used)
rams_used = np.array(rams_used)

cpus_used = cpus_used[np.argsort(date_times_squeue_files_all)]
rams_used = rams_used[np.argsort(date_times_squeue_files_all)]
date_times_squeue_files_all = np.sort(date_times_squeue_files_all)

In [ ]:
fig, ax1 = plt.subplots(dpi=300)
ax1.set_title('Midway3 Partition lgrandi Monitoring')
ax1.set_xlabel('Time')
ax1.tick_params(axis='x', rotation=45)  # Rotate the x-axis labels by 45 degrees

color = 'tab:red'
ax1.set_ylabel('Resource Usage Percentile', color=color)
ax1.plot(date_times_squeue_files_all, cpus_used/256, color=color, alpha=0.5, label='CPUs')
ax1.plot(date_times_squeue_files_all, rams_used/1500, color=color, alpha=0.2, label='Memory')
ax1.tick_params(axis='y', labelcolor=color)
ax1.legend(loc='upper left')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Concurrent Users', color=color)  # we already handled the x-label with ax1
ax2.plot(date_times_squeue_files_all, user_counts, color=color)
ax2.tick_params(axis='y', labelcolor=color)
